In [23]:
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

import warnings



In [24]:
data = pd.read_csv("data/data.csv")
genre_data = pd.read_csv('data/data_by_genres.csv')
year_data = pd.read_csv('data/data_by_year.csv')

### Création de l'authentification API Spotify

In [27]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

SPOTIFY_CLIENT_ID = "ed9ece4a286d42f1aa834c82bed1f95e"
SPOTIFY_CLIENT_SECRET = "e1f250bb36914f3c9235c88df325eff6"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET,
    redirect_uri="http://localhost:8000/callback",
    scope="user-top-read playlist-read-private user-library-read playlist-modify-public playlist-modify-private"
))

### Get the user's top tracks

In [19]:
top_tracks = sp.current_user_top_tracks(limit=50, time_range='medium_term')

print("Your top tracks:")

for idx, track in enumerate(top_tracks['items']):
    print(f"{idx + 1}: {track['name']} by {track['artists'][0]['name']}")
    

Your top tracks:
1: Ma Meilleure Ennemie (from the series Arcane League of Legends) by Stromae
2: Cut the Bridge by Linkin Park
3: Two Faced by Linkin Park
4: Stained by Linkin Park
5: Two Faced by Linkin Park
6: Overflow by Linkin Park
7: The Emptiness Machine by Linkin Park
8: The Emptiness Machine by Linkin Park
9: Heavy Is the Crown by Linkin Park
10: IGYEIH by Linkin Park
11: Over Each Other by Linkin Park
12: Heavy Is the Crown by Linkin Park
13: Good Things Go by Linkin Park
14: 505 by Arctic Monkeys
15: Decode - Twilight Soundtrack Version by Paramore
16: In the End by Linkin Park
17: Pyramid Song by Radiohead
18: The Less I Know The Better by Tame Impala
19: Where Is My Mind? - 2007 Remaster by Pixies
20: No You Girls by Franz Ferdinand
21: WASTE by KXLLSWXTCH
22: Somewhere I Belong by Linkin Park
23: Borderline by Tame Impala
24: Notion by The Rare Occasions
25: A Soul With No King by AURORA
26: CASTLE OF GLASS by Linkin Park
27: Cold Cold Cold by Cage The Elephant
28: BURN I

### Get the track IDs to fetch audio features

In [25]:
track_ids = [track['id'] for track in top_tracks['items']]

print(f"Track IDs: {track_ids}")

Track IDs: ['4lriIG2vNqwDWzOj2I9rtj', '5mFZipkX1HZ4Idz5LOWpzq', '6i5qhmmF9UNUBRyrPSGn4m', '0J1IJsMbKWb5g2sJArTkGF', '5Hh2VJj7bOWGsN0Jw6smwt', '7diOv6wm3ngF7Vt6ubK0Sf', '1EDPVGbyPKJPeGqATwXZvN', '2PnlsTsOTLE5jnBnNe2K0A', '2HBBM75Xv3o2Mqdyh1NcM0', '7M8eZBRTD9QNasEVReEM0H', '3k4gjmYNJugQRlh2aounZ9', '3fgehc497TFqKH1zBL2YNK', '6aCBjSb87RizdH8lVBIRW7', '0BxE4FqsDD1Ot4YuBXwAPp', '6umX518VohIob6IipK3WYM', '60a0Rd6pjrkxjPbaKzXjfq', '55q3Ro66yXWi9rsEddeEN4', '6K4t31amVTZDgR3sKmwUJJ', '7wCmS9TTVUcIhRalDYFgPy', '4VP8QiCeaZq8BeTIUrMQOG', '4NgherJDfCXWBZBYheM4jK', '3fjmSxt0PskST13CSdBUFx', '5hM5arv9KDbCHS0k9uqwjr', '0sTlGEld0h8kIPZaKDYUf4', '3MNSWCyOqjyQVUZCoPKYVL', '1r1fPuhj9H4VdXr7OK6FL5', '1Q3t9fWvHUXKsMmpD2XpUu', '7oVEtyuv9NBmnytsCIsY5I', '2nLtzopw4rPReszdYBJU6h', '0yc6Gst2xkRu0eMLeRMGCX', '0ZucyPms79Cydv0RMYV2Oi', '3qhlB30KknSejmIvZZLjOD', '3aQ9MHkMeL7Yu7jpyF62xn', '3gy21TwRSP40aUMwcY7Rtq', '3K4HG9evC7dg3N0R9cYqk4', '4JHg4nNYUJQ5HULcCmI18R', '7JIuqL4ZqkpfGKQhYlrirs', '3sEufjE0ITaW5qhtnW3avv', 

### Create Spotify playlist

In [29]:
# Get user ID (required to create a playlist)
user_id = sp.current_user()['id']

# Create a new playlist
playlist_name = "AI-ify Playlist"
playlist_description = "This is a playlist created using AI-ify."
playlist = sp.user_playlist_create(user_id, playlist_name, public=True, description=playlist_description)

# Add tracks to the playlist
sp.playlist_add_items(playlist['id'], track_ids)

{'snapshot_id': 'AAAAAnOtBCvDWAOs75w0WfvtnizJmFpR'}